In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import mean_squared_error
import torch.nn.functional as F
from PIL import Image

# Dataset Loader for .npy images
class ROPDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.data = []
        self.labels = []
        self.transform = transform
        self.class_map = {'NORMALROP': 0, 'AROP1': 1, 'AROP2': 2}
        
        # Loop over all subfolders (AROP1, AROP2, NORMALROP)
        for class_name in self.class_map:
            class_dir = os.path.join(root_dir, class_name)
            for file in os.listdir(class_dir):
                if file.endswith('.npy'):
                    img = np.load(os.path.join(class_dir, file))
                    self.data.append(img)
                    self.labels.append(self.class_map[class_name])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        
        # Ensure the image is 2D (grayscale), then expand to 3 channels (RGB)
        if len(img.shape) == 2:  # If it's a grayscale image
            img = np.repeat(img[..., np.newaxis], 3, axis=2)  # Convert to 3 channels (RGB)
        
        if self.transform:
            img = self.transform(img)
        return img, label

# Define transformations (resize and convert to tensor)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),  # Resize image to 224x224 (resnet input size)
])

# Path to preprocessed images
dataset_path = 'Preprocessed_ROP_image'

# Create dataset and DataLoader
dataset = ROPDataset(dataset_path, transform=transform)

# Split dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)

# Model using Transfer Learning (ResNet18)
model = models.resnet18(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, 3)  # Output layer for 3 classes (NORMALROP, AROP1, AROP2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "rop_model.pth")
